# Ibraheem Al-Yousef PHY373

## Q1) 
### 1- Reading Vesta:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the data
sc = np.loadtxt("SC.txt", skiprows=1)
fcc = np.loadtxt("FCC.txt", skiprows=1)
bcc = np.loadtxt("BCC.txt", skiprows=1)
# Extract the columns for 2Theta and intensity
theta = np.linspace(0, 180, num=int(180 / 0.001) + 1)
intensity_sc = np.zeros_like(theta)
intensity_bcc = np.zeros_like(theta)
intensity_fcc = np.zeros_like(theta)

for i in range(len(sc[:, 7])):
    index = (np.abs(theta - (sc[:, 7])[i])).argmin()
    intensity_sc[index] = sc[:, 8][i]


for i in range(len(fcc[:, 7])):
    index = (np.abs(theta - (fcc[:, 7])[i])).argmin()
    intensity_fcc[index] = fcc[:, 8][i]

intensity = np.zeros_like(theta)
for i in range(len(bcc[:, 7])):
    index = (np.abs(theta - (bcc[:, 7])[i])).argmin()
    intensity_bcc[index] = bcc[:, 8][i]


plt.plot(theta, intensity_sc + 220, color="r")
plt.plot(theta, intensity_fcc + 110, color="g")
plt.plot(theta, intensity_bcc, color="b")
plt.xlabel("2θ°")
plt.ylabel("Intensity")
plt.legend(["SC", "FCC", "BCC"])
plt.xlim(0, 90)
plt.yticks([])
plt.show()

# Create a figure with three subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True)

# Plot the first set of data on the first subplot
ax1.plot(theta, intensity_sc, color="r")
ax1.set_xlim(0, 90)

# Plot the second set of data on the second subplot
ax2.plot(theta, intensity_fcc, color="g")
ax2.set_ylabel("Intensity")
ax2.set_xlim(0, 90)

# Plot the third set of data on the third subplot
ax3.plot(theta, intensity_bcc, color="b")
ax3.set_xlim(0, 90)

fig.text(0.5, 0.04, "2θ°", ha="center")
fig.legend(["SC", "FCC", "BCC"])
plt.subplots_adjust(hspace=0.0)
plt.show()

### 2- Generating the XRD pattern:

In [ ]:
# Define the lattice constant and atomic scattering factor for Mg
a = 4.5  # angstroms
wavelength = 1.54059
millet_max = 5

# Generate all possible combinations of h, k, and l
hkl = (
    np.mgrid[0 : millet_max + 1, 0 : millet_max + 1, 0 : millet_max + 1]
    .reshape(3, -1)
    .T
)

# Select only the combinations that satisfy the Bcc reflection condition (h+k+l is even)
G = hkl[np.sum(hkl, axis=1) % 2 == 0][1::]

d = [a / np.sqrt(G[i][0] ** 2 + G[i][1] ** 2 + G[i][2] ** 2) for i in range(len(G))]
two_theta = [2 * np.arcsin(wavelength / (2 * d[i])) * 180 / np.pi for i in range(len(d)) if ~np.isnan(2 * np.arcsin(wavelength / (2 * d[i])) * 180 / np.pi)]

Fs = [G[i][0] + G[i][1] + G[i][2] for i in range(len(G))]
F = [1 + (-1) ** Fs[i] for i in range(len(Fs))]
I = [np.abs(F[i]) ** 2 for i in range(len(F))]

theta = np.linspace(0, 180, num=int(180 / 0.001) + 1)
intensity = np.zeros_like(theta)
for i in range(len(two_theta)):
    index = (np.abs(theta - two_theta[i])).argmin()
    intensity[index] = I[i]
plt.xlabel("2θ°")
plt.ylabel("Intensity")
plt.xlim(0, 90)
plt.yticks([])
plt.plot(theta, intensity_bcc / 100 + 2, color="b")
plt.plot(theta, intensity / 4, color="g")
plt.legend(["BCC Vesta", "BCC Me (Python)"])
plt.show()

### 3- Present and Absent planes:

In [ ]:
absent = sorted(hkl[np.sum(hkl, axis=1) % 2 == 1], key=lambda x: np.linalg.norm(x))
present = sorted(
    hkl[np.sum(hkl, axis=1) % 2 == 0][1::], key=lambda x: np.linalg.norm(x)
)
print("The first 10 present planes are:")
print("%s %5s %5s" % ("h", "k", "l"))
for i in range(10):
    print("%i %5i %5i" % (present[i][2], present[i][1], present[i][0]))

print("\n\nThe first 5 absent planes are:")
print("%s %5s %5s" % ("h", "k", "l"))
for i in range(5):
    print("%i %5i %5i" % (absent[i][2], absent[i][1], absent[i][0]))

## Q2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def simple_pendulum(
    theta0, omega0, L=1.0, m=1.0, g=9.81, dt=0.01, N=1000, method="euler", plot=True
):
    """
    Computes the angle, angular velocity, and potential energy of a simple pendulum using the Euler method.

    Parameters:
    ----------
    theta0 : float
        Initial angle (in radians).
    omega0 : float
        Initial angular velocity (in radians per second).
    L : float, optional
        Length of pendulum (in meters). Default is 1.0.
    m : float, optional
        Mass of pendulum bob (in kilograms). Default is 1.0.
    g : float, optional
        Acceleration due to gravity (in meters per second squared). Default is 9.81.
    dt : float, optional
        Time step for Euler method (in seconds). Default is 0.01.
    N : int, optional
        Number of iterations for Euler method. Default is 1000.
    plot : bool, optional
        Whether to plot the results. Default is True.

    Returns:
    -------
    theta : numpy array
        Array of angles (in radians) at each time step.
    omega : numpy array
        Array of angular velocities (in radians per second) at each time step.
    t : numpy array
        Array of times (in seconds) at each time step.
    U : numpy array
        Array of potential energies (in Joules) at each time step.
    """

    # Define arrays to store results
    theta = np.zeros(N)
    omega = np.zeros(N)
    alpha = np.zeros(N)
    t = np.zeros(N)
    U = np.zeros(N)
    K = np.zeros(N)
    # Apply Euler method to solve ODEs

    if method == "euler":
        theta[0] = theta0
        omega[0] = omega0
        t[0] = 0.0
        U[0] = m * g * L * (1 - np.cos(theta0))
        K[0] = 0.5 * m * L**2 * omega0**2

        for i in range(1, N):
            theta[i] = theta[i - 1] + omega[i - 1] * dt
            omega[i] = omega[i - 1] - (g / L) * np.sin(theta[i - 1]) * dt
            t[i] = t[i - 1] + dt
            U[i] = m * g * L * (1 - np.cos(theta[i]))
            K[i] = 0.5 * m * L**2 * omega[i] ** 2

    elif method == "euler-cromer":
        theta[0] = theta0
        omega[0] = omega0
        t[0] = 0.0
        U[0] = m * g * L * (1 - np.cos(theta0))
        K[0] = 0.5 * m * L**2 * omega0**2

        for i in range(1, N):
            omega[i] = omega[i - 1] - (g / L) * np.sin(theta[i - 1]) * dt
            theta[i] = theta[i - 1] + omega[i] * dt
            t[i] = t[i - 1] + dt
            U[i] = m * g * L * (1 - np.cos(theta[i]))
            K[i] = 0.5 * m * L**2 * omega[i] ** 2

    elif method == "euler-verlet":
        theta[0] = theta0
        omega[0] = omega0
        alpha[0] = -(g / L) * np.sin(theta0)
        t[0] = 0.0
        U[0] = m * g * L * (1 - np.cos(theta0))
        K[0] = 0.5 * m * L**2 * omega0**2

        for i in range(1, N):
            theta[i] = theta[i - 1] + omega[i - 1] * dt + 0.5 * alpha[i - 1] * dt**2
            alpha[i] = -(g / L) * np.sin(theta[i])
            omega[i] = omega[i - 1] + 0.5 * (alpha[i] + alpha[i - 1]) * dt
            t[i] = t[i - 1] + dt
            U[i] = m * g * L * (1 - np.cos(theta[i]))
            K[i] = 0.5 * m * L**2 * omega[i] ** 2
    # Plot results
    if plot:
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 16))

        ax1.plot(t, theta, label=r"$\theta$", color="r")
        ax1.plot(t, omega, label=r"$\omega$", color="b")
        ax1.legend()
        ax1.set_xlabel("Time (s)")
        ax1.set_ylabel("Angle (rad) / Angular Velocity (rad/s)")
        ax1.set_title(
            "Theta and Omega vs Time for dt =" + str(dt) + " and " + method + " method"
        )

        ax2.plot(theta, omega, color="k")
        ax2.set_xlabel("Angle (rad)")
        ax2.set_ylabel("Angular Velocity (rad/s)")
        ax2.set_title(
            "Theta vs Omega for dt =" + str(dt) + " and " + method + " method"
        )

        ax3.plot(t, U, label="Potential Energy", color="r")
        ax3.plot(t, K, label="Kinetic Energy", color="b")
        ax3.plot(t, U+K, label="Total Energy", color="k")
        ax3.legend()
        ax3.set_xlabel("Time (s)")
        ax3.set_ylabel("Energy (J)")
        ax3.set_title(
            "Energy vs Time for dt =" + str(dt) + " and " + method + " method"
        )

        plt.tight_layout()
        plt.show()
    else:
        return theta, omega, t, U, K

In [ ]:
methods = ["euler", "euler-cromer", "euler-verlet"]
times = [[1, 20], [0.5, 40], [0.01, 2000]]
for time in times:
    for methoda in methods:
        simple_pendulum(theta0=0.15, omega0=2.0, dt=time[0], N=time[1], method=methoda)